In [1]:
import csv
from collections import defaultdict
import json
import os


import random

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from energy_py.common.utils import print_txt

from energy_py.common.experiments.analysis import read_run_results

/Users/adam/anaconda3/envs/energy_py/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)
/Users/adam/anaconda3/envs/energy_py/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:


def process_result(result, useful_cols, verbose=False):
    result = result[useful_cols]

    total_reward = result['reward'].sum()
#     num_actions = result['flex_counter'].max()

    output = {
        #'number flex actions': num_actions,
        'total episode reward [$]': total_reward
    }

    if verbose:
        groups = {}
        groups['electricity_price'] = [np.mean, np.max, np.min]
        groups['flex_up'] = [np.sum]
        groups['flex_down'] = [np.sum]
        groups['flex_up'] = [np.sum]
        
        group = result.groupby('flex_counter').agg(groups)
        print(group)

        for k, v in output.items():
            print('{} {:2.0f}'.format(k, v))

    return output



class Results(object):
    """
    Processes environment histories
    
    args
        run_name (path) /expt_name/run_name
        verbose (bool)
    """
    def __init__(self, run_name, cols, verbose=False):
        self.run_name = run_name
        self.cols = cols
        
        self.results = read_run_results(self.run_name, verbose)
        
        self.episode_length = self.results[-1].shape[0]
        
        self.episode_summary = self.summarize_results()
        
    def summarize_results(self):
        result_summary = defaultdict(list)

        for result in self.results:
            out = process_result(result, self.cols)
            for k, v in out.items():
                result_summary[k].append(v)
                
        episode_summary = pd.DataFrame().from_dict(result_summary)
        
        num_5mins_per_day = 12 * 24
        num_5mins_per_year = num_5mins_per_day * 365
        
        avg_ep_reward = episode_summary['total episode reward [$]'].mean()
        avg_daily_reward = avg_ep_reward / (self.episode_length / num_5mins_per_day)
        avg_annual_reward = avg_ep_reward / (self.episode_length / num_5mins_per_year)

        print('Results for {}'.format(self.run_name))
        print(' ')
        print('avg episode reward [$] {:2.0f}'.format(avg_ep_reward))
        print('avg daily reward [$] {:2.0f}'.format(avg_daily_reward))
        print('avg annual reward [$] {:2.0f}'.format(avg_annual_reward))

        #num_eps_no_flex = episode_summary[episode_summary['number flex actions'] == 0].shape[0]
        #print('num episodes without flexing {} of {} days'.format(num_eps_no_flex, episode_summary.shape[0]))
        
        num_eps_losses = episode_summary[episode_summary['total episode reward [$]'] < 0].shape[0]
        print('num episodes of losses {}'.format(num_eps_losses))
        
        all_results = pd.concat(self.results, axis=0)
        monthly = all_results['reward'].groupby(all_results.index.month).sum()
        print(monthly)
        
        return out

In [3]:
res = Results('new_flex/autoflex', 
              ['charge', 
               'electricity_price', 
               'reward',
               'site_demand',
               'site_electricity_consumption'
              ], 
              verbose=True)

Reading results for new_flex/autoflex
 
AGENT ARGS
agent_id,autoflex
act_path,/Users/adam/git/energy_py/energy_py/experiments/results/new_flex/tensorboard/autoflex/act
total_steps,1
learn_path,/Users/adam/git/energy_py/energy_py/experiments/results/new_flex/tensorboard/autoflex/learn
env_repr,<energy_py flex environment>
seed,2
env,<energy_py flex environment>
sess,<tensorflow.python.client.session.Session object at 0x120320978>

ENV ARGS
episode_length,0
dataset,example
episode_sample,full
env_id,flex



NameError: name 'os' is not defined

In [ ]:
def plot_result(hist, cols):
    hist = hist.iloc[-200:, :]
    f, axes = plt.subplots(nrows=len(cols), 
                           figsize=(len(cols)*5, len(cols)*4))
    axes = axes.flatten()
    
    for ax, col in zip(axes, cols):
        hist.plot(y=col, color='g', ax=ax)
        ax.set_title(col)

    return f
        

_ = plot_result(res.results[0], res.cols)

In [ ]:
res.results[0]['action'].describe()

In [ ]:
res = Results('new_flex/random', 
              ['charge', 
               'electricity_price', 
               'reward',
               'site_demand',
               'site_electricity_consumption'
              ], 
              verbose=True)